In [1]:
import os
os.environ["USE_TF"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [2]:

print("TRANSFORMERS_NO_TF =", os.environ.get("TRANSFORMERS_NO_TF"))

TRANSFORMERS_NO_TF = 1


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [4]:
#data laoding and formatting
from datasets import load_dataset
dataset = load_dataset("json", data_files=r"C:\Users\Admin\Downloads\mini_email_dataset.json")

In [6]:
#inspecting sample from the loaded dataset
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20
    })
})
{'instruction': 'Convert the input into a crisp professional email with a subject line.', 'input': 'Tell Derrick there are still surrendered items in the report. Ask him to check again.', 'output': 'Subject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian'}


In [7]:
#formatting
def format(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:{example['input']}\n\n### Response:\n{example['output']}"
    return{'text' : prompt}

In [8]:
#applying the formatting to the entire dataset
formatted_dataset = dataset['train'].map(format)

In [10]:
print(formatted_dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 20
})


In [11]:
#defining tokenization function
def tokenize_function(example):
    return tokenizer(
        example['text'],
        padding = 'max_length',
        truncation = True,
        max_length = 512,
        return_tensors = 'pt'
    )

In [12]:
#appying the tokenizer
tokenized_dataset = formatted_dataset.map(tokenize_function, batched = True)

In [13]:
print(tokenized_dataset[0])

{'instruction': 'Convert the input into a crisp professional email with a subject line.', 'input': 'Tell Derrick there are still surrendered items in the report. Ask him to check again.', 'output': 'Subject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian', 'text': '### Instruction:\nConvert the input into a crisp professional email with a subject line.\n\n### Input:Tell Derrick there are still surrendered items in the report. Ask him to check again.\n\n### Response:\nSubject: Review of Surrendered Items\n\nHi Derrick,\n\nThere are still surrendered items showing in the report. Please go through it again and ensure everything is properly accounted for.\n\nBest regards,\nBrian', 'input_ids': [21017, 46486, 25, 198, 3103, 1851, 262, 5128, 656, 257, 23453, 4708, 3053, 351, 257, 2426, 1627, 13, 198, 198, 21017, 23412, 25, 24446, 32979, 6

In [25]:
pip install -U "accelerate>=0.26.0"

Note: you may need to restart the kernel to use updated packages.


In [14]:
#fine config
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./distilgpt2-email-finetuned",
    per_device_train_batch_size = 1,
    num_train_epochs = 4,
    logging_steps = 10,
    save_steps = 10,
    save_total_limit=2,                
    fp16=False
)

In [18]:

from transformers import DataCollatorForLanguageModeling


In [19]:
#plugging the trainer
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Because GPT2 is a causal LM, not a masked LM
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [21]:
#start training
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.653600
20,2.781400
30,2.194800
40,2.085300
50,1.744200
60,1.611200
70,1.514600
80,1.486900


TrainOutput(global_step=80, training_loss=2.1340031743049623, metrics={'train_runtime': 234.8985, 'train_samples_per_second': 0.341, 'train_steps_per_second': 0.341, 'total_flos': 10451870023680.0, 'train_loss': 2.1340031743049623, 'epoch': 4.0})